In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [2]:
from cleaner import new_clean
import pandas as pd
import numpy as np

## Reading and Preparing Train Data

The modifications from Approach 2 to Approach 3, as described in the report are all handled in our cleaner function *new_clean()*

In [45]:
df = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

## Train LightGBM model

In [46]:
df = new_clean(df)
test = new_clean(test,is_test=True)

In [47]:
X = df.drop(['price'],axis=1)
Y = df.price

In [48]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=42)

In [50]:
model = lgb.LGBMRegressor(
    boosting_type="gbdt",
    num_iterations = 2500,
    learning_rate = 0.05,
    num_leaves=15,
    tree_learner='feature',
    max_depth =10,
    min_data_in_leaf=7,
    bagging_fraction = 1,
    bagging_freq = 100,
    reg_sqrt='True',
    metric ='rmse',
    feature_fraction = 0.6,
    random_state=42)

model.fit(X_train,y_train) 


preds = model.predict(X_test)
rmse_lgb = mean_squared_error(y_test, preds,squared = False)
print(" RMSE: %f" % (rmse_lgb))

/Users/kanav7/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] bagging_fraction is set=1, subsample=1.0 will be ignored. Current value: bagging_fraction=1
[LightGBM] [Warning] min_data_in_leaf is set=7, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=7
[LightGBM] [Warning] bagging_freq is set=100, subsample_freq=0 will be ignored. Current value: bagging_freq=100
 RMSE: 23222.070367


## Train CatBoost model

In [51]:
#TODO

### Predict Values for Test Data

In [52]:
test['Predicted'] = model.predict(test)

In [53]:
test['id'] = test.index

In [54]:
test.Predicted = test.Predicted.apply(lambda x:round(x / 100.0) * 100.0)

In [55]:
submission = test[['id','Predicted']]

In [56]:
submission

,id,Predicted
0,0,152300.0
1,1,264800.0
2,2,209400.0
3,3,164900.0
4,4,64700.0
...,...,...
4995,4995,71500.0
4996,4996,158600.0
4997,4997,56200.0
4998,4998,67800.0


In [53]:
submission.to_csv('data/submission.csv',index=False)